In [1]:
import pandas as pd

In [2]:
reviews_data = pd.read_csv('E:/Data-Deep-Learning/Reviews.csv')

In [3]:
reviews_data.to_pickle('E:/Data-Deep-Learning/review_data.pkl')

In [4]:
reviews_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
reviews_data = reviews_data[['Score','Text']]

In [6]:
reviews_data = reviews_data[reviews_data.Score != 3]

In [7]:
reviews_data.head()

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [8]:
reviews_data['Sentiment'] = reviews_data.Score.map(lambda x: 1 if x > 3 else 0)

In [9]:
reviews_data.head()

,Score,Text,Sentiment
0,5,I have bought several of the Vitality canned d...,1
1,1,Product arrived labeled as Jumbo Salted Peanut...,0
2,4,This is a confection that has been around a fe...,1
3,2,If you are looking for the secret ingredient i...,0
4,5,Great taffy at a great price. There was a wid...,1


### Understanding data distribution

In [10]:
reviews_data.Sentiment.value_counts()

1    443777
0     82037
Name: Sentiment, dtype: int64

In [11]:
 from sklearn.model_selection import train_test_split

In [12]:
Y = pd.get_dummies(reviews_data.Sentiment).values

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM
from keras.layers.core import SpatialDropout1D

C:\Workspace\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
max_words = 2000
tokenizer = Tokenizer(num_words=max_words,split=' ')

In [15]:
tokenizer.fit_on_texts(reviews_data.Text)

In [16]:
X = tokenizer.texts_to_sequences(reviews_data.Text)

In [17]:
X = pad_sequences(X,maxlen=1000)

In [18]:
trainX,testX,trainY,testY = train_test_split(X,Y)

### Model Creation

In [19]:
model=  Sequential()

In [20]:
model.add(Embedding(max_words, 128, input_length=X.shape[1]))

In [21]:
model.add(SpatialDropout1D(0.4))

In [22]:
model.add(LSTM(196,dropout=0.2,recurrent_dropout=0.2))

In [23]:
model.add(Dense(2,activation='softmax'))

In [24]:
from sklearn.utils import class_weight
import numpy as np

In [25]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(reviews_data.Sentiment),reviews_data.Sentiment)

In [26]:
class_weights

array([3.20473689, 0.59243043])

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 128)         256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1000, 128)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(trainX,trainY, epochs=10, b
          atch_size=28,validation_split=0.2, class_weight=class_weights)

Train on 315488 samples, validate on 78872 samples
Epoch 1/10
315476/315488 [============================>.] - ETA: 0s - loss: 0.1965 - acc: 0.9224

In [ ]:
model.save('model-lstm.h5')

In [ ]:
model.fit(trainX,trainY, epochs=5, batch_size=28,validation_split=0.2, class_weight=class_weights)
model.save('model-lstm-2.h5')